In [10]:
import warnings
warnings.filterwarnings("ignore")
from LLM_auto_label import get_response_from_prompt
from langchain_community.tools import TavilySearchResults
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from uqlm import BlackBoxUQ
from uqlm import UQEnsemble
from uqlm import LLMPanel
from uqlm.judges import LLMJudge
import os
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import tqdm
from collections import Counter
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
os.environ["GEMINI_API_KEY"] = GOOGLE_API_KEY
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")
os.environ["TOGETHER_API_KEY"] = TOGETHER_API_KEY
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [11]:
dataset_src = "New Dataset/"
csv_files = [f for f in os.listdir(dataset_src) if f.endswith('.csv')]
csv_files

['Albert Einstein.csv',
 'Bill Gates.csv',
 'Bruno Mars.csv',
 'Elon Musk.csv',
 'Hoài Linh.csv',
 'Nam Cao.csv',
 'Nguyễn Du.csv',
 'Ngô Bảo Châu.csv',
 'Phạm Nhật Vượng.csv',
 'Steve Jobs.csv',
 'Sơn Tùng M-TP.csv',
 'Taylor Swift.csv']

In [12]:
df_list = []
for csv in csv_files:
	entity_name = csv.split('.')[0]
	df = pd.read_csv(f"{dataset_src}{csv}")
	df['entity'] = entity_name
	df_list.append(df)

df = pd.concat(df_list, ignore_index=True)
df.head()

,atomic_facts,label,search_results,prompt,entity,example_response
0,Einstein là một nhà vật lý lý thuyết nổi tiếng...,Supported,[{'title': 'Albert Einstein – Wikipedia tiếng ...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Albert Einstein,NaN
1,Einstein được biết đến rộng rãi với những đóng...,Supported,[{'title': 'Albert Einstein: Từ cậu bé chậm nó...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Albert Einstein,NaN
2,Einstein đặc biệt nổi tiếng với thuyết tương đối.,Supported,[{'title': 'Thuyết tương đối – Wikipedia tiếng...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Albert Einstein,NaN
3,Einstein sinh ngày 14 tháng 3 năm 1879 tại Ulm.,Supported,[{'title': 'Albert Einstein – Wikipedia tiếng ...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Albert Einstein,NaN
4,Ulm thuộc Vương quốc Württemberg.,Supported,[{'title': 'Albert Einstein sinh ngày 14 tháng...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Albert Einstein,NaN


In [13]:
df = df[:2]
df.head()

,atomic_facts,label,search_results,prompt,entity,example_response
0,Einstein là một nhà vật lý lý thuyết nổi tiếng...,Supported,[{'title': 'Albert Einstein – Wikipedia tiếng ...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Albert Einstein,NaN
1,Einstein được biết đến rộng rãi với những đóng...,Supported,[{'title': 'Albert Einstein: Từ cậu bé chậm nó...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Albert Einstein,NaN


In [14]:
atomic_facts = df['atomic_facts'].tolist()
atomic_facts

['Einstein là một nhà vật lý lý thuyết nổi tiếng người Đức.',
 'Einstein được biết đến rộng rãi với những đóng góp đột phá trong lĩnh vực vật lý.']

In [15]:
labels = df['label'].tolist()
labels

['Supported', 'Supported']

In [16]:
prompts = df['prompt'].to_list()
prompts

["\nBạn được cung cấp một ngữ cảnh (kết quả tìm kiếm web) kèm theo một atomic fact. Nhiệm vụ của bạn là gán nhãn cho atomic fact này là Supported, Unsupported, hoặc Irrelevant. Ngữ cảnh nói về một người hoặc một nhân vật và atomic fact là một thông tin về người đó.\n\nSupported: Chọn nhãn này nếu atomic fact là đúng và được ngữ cảnh hoàn toàn xác thực.\nUnsupported: Chọn nhãn này nếu atomic fact là sai và mâu thuẫn với ngữ cảnh.\nIrrelevant: Chọn nhãn này nếu atomic fact không liên quan đến người hoặc nhân vật, hoặc atomic fact nói về một điều khác không có liên hệ với người đó.\n\nLƯU Ý: thông tin tìm kiếm được trên trang web có thể chứa tin giả và tin sai sự thật. Hãy cân nhắc kĩ lưỡng trước khi ra quyết định của mình.\nĐây là phần bạn cần làm:\nNhân vật: Albert Einstein\nNgữ cảnh từ kết quả tìm kiếm web: [{'title': 'Albert Einstein – Wikipedia tiếng Việt', 'url': 'https://vi.wikipedia.org/wiki/Albert_Einstein', 'content': 'Albert Einstein (14 tháng 3 năm 1879 – 18 tháng 4 năm 1955) 

In [17]:
class LLMFactory:
    def __init__(self, model_name, api_key=None, together=False):
        self.model_name = model_name
        self.api_key = api_key
        self.together = together
        self.llm = self._initialize_llm()

    def _initialize_llm(self):
        if self.together:
            base_url = "https://api.together.xyz/v1"
            api_key = self.api_key or TOGETHER_API_KEY
            if not api_key:
                raise ValueError("Together.ai API key not found.")
            return ChatOpenAI(api_key=api_key, base_url=base_url, model=self.model_name)

        if "gpt" in self.model_name:
            if not self.api_key:
                self.api_key = os.getenv("OPENAI_API_KEY")
            if not self.api_key:
                raise ValueError("OpenAI API key not found.")
            return ChatOpenAI(api_key=self.api_key, model=self.model_name)
        
        elif "gemini" in self.model_name:
            if not self.api_key:
                self.api_key = os.getenv("GEMINI_API_KEY")
            if not self.api_key:
                raise ValueError("Google API key not found.")
            return ChatGoogleGenerativeAI(google_api_key=self.api_key, model=self.model_name)
        
        elif "claude" in self.model_name:
            if not self.api_key:
                self.api_key = os.getenv("ANTHROPIC_API_KEY")
            if not self.api_key:
                raise ValueError("Anthropic API key not found.")
            return ChatAnthropic(anthropic_api_key=self.api_key, model_name=self.model_name)

        elif "Qwen" in self.model_name:
            base_url = "https://api.together.xyz/v1"
            api_key = self.api_key or TOGETHER_API_KEY
            if not api_key:
                raise ValueError("Together.ai API key not found for Qwen model.")
            return ChatOpenAI(api_key=api_key, base_url=base_url, model=self.model_name)
        elif "Llama" in self.model_name:
            base_url = "https://api.together.xyz/v1"
            api_key = self.api_key or TOGETHER_API_KEY
            if not api_key:
                raise ValueError("Together.ai API key not found for Llama model.")
            return ChatOpenAI(api_key=api_key, base_url=base_url, model=self.model_name)
        elif "deepseek" in self.model_name:
            base_url = "https://api.together.xyz/v1"
            api_key = self.api_key or TOGETHER_API_KEY
            if not api_key:
                raise ValueError("Together.ai API key not found for DeepSeek model.")
            return ChatOpenAI(api_key=api_key, base_url=base_url, model=self.model_name)
        else:
            raise ValueError(f"Unsupported LLM model: {self.model_name}")

    def get_llm(self):
        return self.llm

In [18]:
# Define model names for easy switching
# model_gpt = "gpt-4o-mini"
# model_gemini = "gemini-2.0-flash"
model_claude = "claude-3-haiku-20240307"

# Initialize 3 LLMs (can be any combination)
# llm_factory_1 = LLMFactory(model_name=model_gpt, together=False)
# llm_factory_2 = LLMFactory(model_name=model_gemini, together=False)
llm_factory_3 = LLMFactory(model_name=model_claude, together=False)
# llm1 = llm_factory_1.get_llm()
# llm = llm_factory_2.get_llm()
llm = llm_factory_3.get_llm()

predicted_labels = []
full_responses = []
correct_prediction = []
for label, atomic_fact,prompt in zip(labels,atomic_facts,prompts):
	print(f"Processing atomic fact: {atomic_fact}")
	print(f"Label: {label}")
	result = get_response_from_prompt(prompt,llm)
	print(result)
	predicted_labels.append(result['label'])
	print(f"Predicted label: {result['label']}")
	full_responses.append(result['full_response'])
	if label==result['label']:
		correct_prediction.append(True)
	else:
		correct_prediction.append(False)

print(correct_prediction)

Processing atomic fact: Einstein là một nhà vật lý lý thuyết nổi tiếng người Đức.
Label: Supported
{'full_response': 'Label: Supported\nReason: The provided context clearly states that Albert Einstein was a famous theoretical physicist from Germany, which supports the given atomic fact.', 'label': 'Supported', 'prompt': "\nBạn được cung cấp một ngữ cảnh (kết quả tìm kiếm web) kèm theo một atomic fact. Nhiệm vụ của bạn là gán nhãn cho atomic fact này là Supported, Unsupported, hoặc Irrelevant. Ngữ cảnh nói về một người hoặc một nhân vật và atomic fact là một thông tin về người đó.\n\nSupported: Chọn nhãn này nếu atomic fact là đúng và được ngữ cảnh hoàn toàn xác thực.\nUnsupported: Chọn nhãn này nếu atomic fact là sai và mâu thuẫn với ngữ cảnh.\nIrrelevant: Chọn nhãn này nếu atomic fact không liên quan đến người hoặc nhân vật, hoặc atomic fact nói về một điều khác không có liên hệ với người đó.\n\nLƯU Ý: thông tin tìm kiếm được trên trang web có thể chứa tin giả và tin sai sự thật. Hãy 

In [21]:
print("Processing black box")
bbuq = BlackBoxUQ(llm=llm, scorers=["semantic_negentropy"], use_best=True, max_calls_per_min=5)

results_BB = await bbuq.generate_and_score(prompts=prompts, num_responses=5)

# LLM as a judge
print("Processing LLM as a Judge")
panel = LLMPanel(llm=llm, judges=[llm,llm,llm,llm,llm], scoring_templates=['true_false_uncertain','true_false_uncertain','true_false_uncertain','true_false_uncertain','true_false_uncertain'],max_calls_per_min=5)
results_judge = await panel.generate_and_score(prompts=prompts)

Processing black box


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Generating responses...
Generating candidate responses...
Computing confidence scores...
Processing LLM as a Judge
Generating responses...
Generating LLMJudge scores...
Generating LLMJudge scores...
Generating LLMJudge scores...
Generating LLMJudge scores...
Generating LLMJudge scores...


In [ ]:
results_BB.to_df()

,response,sampled_responses,prompt,semantic_negentropy
0,Label: Supported\nReason: Các kết quả tìm kiếm...,[Label: Supported\nReason: Các kết quả tìm kiế...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,0.644755
1,Label: Supported\nReason: Ngữ cảnh cung cấp th...,[Label: Supported\nReason: Các đoạn văn bản tr...,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,1.000000


In [ ]:
results_judge.to_df()

,prompt,response,judge_1,judge_2,judge_3,judge_4,judge_5,avg,max,min,median
0,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Label: Supported\nReason: Ngữ cảnh cung cấp th...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,\nBạn được cung cấp một ngữ cảnh (kết quả tìm ...,Label: Supported\nReason: Các đoạn văn bản tro...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
result_BB = results_BB.to_df()
result_judge = results_judge.to_df()
result =((result_judge['avg'] + result_BB['semantic_negentropy'])/2)>= 0.80

In [ ]:
# result = results.to_df()
# result['ensemble_score'] = result['ensemble_score'] >= 0.90
confidence_list = result.tolist()
for i in range(len(confidence_list)):
	if confidence_list[i]==True and correct_prediction[i]==False:
		print("Wrong atomic fact:", atomic_facts[i])

print(confidence_list)
print(correct_prediction)

[True, True]
[True, True]


In [ ]:
count = 0
for v in confidence_list:
	if v == False:
		count+=1
print("Check",count)
print("Total", len(confidence_list))

Check 0
Total 2
